In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

cities_path = "../WeatherPy/cities_weather.csv"

# Read School and Student Data File and store into Pandas DataFrames
cities_data = pd.read_csv(cities_path)

#Create the dataframe
cities_data_df = pd.DataFrame(cities_data)
cities_data.dtypes

Cities                  object
Countries               object
Latitude               float64
Longitude              float64
Max Temperature (F)    float64
Humidity (%)             int64
Cloudiness               int64
Wind Speed (mph)       float64
Date                    object
dtype: object

In [3]:
#Humidity Heatmap

#Configure gmaps
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_data_df[["Latitude", "Longitude"]]
weights = cities_data_df["Humidity (%)"]

#Create map
figure_layout = {'width': '800px',
                 'height': '600px',
                 'border': '1px solid black',
                 'padding': '1px',
                 'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, center=(0,0), zoom_level=1)

In [4]:
#Add Heatmap layer to map.
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [13]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.

#Max temp between 75 and 90 degrees F.
#Wind speed between 1 and 15 mph.
#Cloudiness between 10 and 50%

for index, rows in cities_data_df.iterrows():
    if cities_data_df.loc[index, "Max Temperature (F)"] < 90:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
    if cities_data_df.loc[index, "Max Temperature (F)"] >= 75:
            pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue 
    if cities_data_df.loc[index, "Wind Speed (mph)"] <= 15:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
    if cities_data_df.loc[index, "Cloudiness"] <= 40:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
    if cities_data_df.loc[index, "Cloudiness"] >= 5:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
        
cities_data_df        

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


,Cities,Countries,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Date
0,NaN,SN,14.90,-12.46,72.5,20,80,10.5,02-09-2021
1,NaN,SH,-15.94,-5.72,68.4,88,54,15.0,02-09-2021
2,NaN,RU,59.70,30.79,5.0,85,90,2.2,02-09-2021
3,NaN,BR,-22.97,-42.03,72.4,73,100,2.1,02-09-2021
4,NaN,CA,50.70,-127.42,37.4,74,1,8.0,02-09-2021
5,NaN,PF,-9.80,-139.03,79.6,78,99,18.8,02-09-2021
6,NaN,AU,-35.02,117.89,71.6,49,75,17.3,02-09-2021
7,The Valley,AI,18.22,-63.06,78.8,69,20,12.7,02-09-2021
8,NaN,AR,-54.80,-68.30,55.4,62,75,16.1,02-09-2021
9,NaN,AU,-35.72,150.18,71.0,70,80,3.0,02-09-2021


In [15]:
# Drop any rows will null values.

cities_data_df.dropna(inplace=True)
cities_data_df.reset_index(drop=True, inplace=True)
cities_data_df

,Cities,Countries,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Date
0,The Valley,AI,18.22,-63.06,78.8,69,20,12.7,02-09-2021
1,Olinda,BR,-8.01,-34.86,78.8,69,40,6.9,02-09-2021
2,Moree,AU,-29.47,149.85,86.0,32,40,10.4,02-09-2021
